In [1]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
#getting the data from the directory
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Helper Function

In [45]:
def init_wt(shape):
    init_random_dist = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [7]:
def init_base (shape):
    init_base = tf.constant(0.1,shape=shape)
    return tf.Variable(init_base)

In [9]:
def conv2d (x, W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding= "SAME")

In [10]:
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1], padding= "SAME")

In [33]:
def convolution_Layer (input_x, shape):
    W = init_wt(shape)
    b = init_base([shape[3]])
    return tf.nn.relu(conv2d(input_x, W)+b)

In [34]:
# full Normal layer
def normal_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_wt([input_size, size])
    b = init_base([size])
    return tf.matmul(input_layer, W) + b

# Placeholder and normal Tensorflow operation

In [28]:
X = tf.placeholder(dtype=tf.float32,shape=[None,784])
y_true = tf.placeholder(dtype=tf.float32,shape=[None,10])

In [29]:
#Layers

X_image = tf.reshape(X, shape=[-1,28,28,1])


In [35]:
#Convulution 1 layer
conv1_layer = convolution_Layer(X_image,shape=[6,6,1,32])
conv1_pool = max_pool(conv1_layer)

In [37]:
#convolution layer 2
conv2_layer = convolution_Layer(conv1_pool,shape=[6,6,32,64])
conv2_pool = max_pool(conv2_layer)

In [38]:
#final Layer

conv_flat = tf.reshape(conv2_pool,shape=[-1,7*7*64])
full_layer = tf.nn.relu(normal_layer(conv_flat,size=1024))


In [39]:
#Hold on Prob
hold_prob = tf.placeholder(dtype=tf.float32)
full_one_dropout = tf.nn.dropout(x=full_layer,keep_prob=hold_prob)

In [40]:
y_pred = normal_layer(full_one_dropout,10)

In [59]:
#optimie
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
optim = tf.train.AdamOptimizer(learning_rate=0.001)
train = optim.minimize(cross_entropy)

In [61]:
#initialoise global variable
init = tf.global_variables_initializer()

In [62]:
#Processing on CPU

with tf.Session() as sess:
    steps = 5000
    sess.run(init)
    for step in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict={X:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if step%100 ==0:
            
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,dtype=tf.float32))
            
            print(sess.run(acc,feed_dict = {X : mnist.test.images, y_true : mnist.test.labels,hold_prob:1.0}))
    
    

0.1037
0.855
0.9097
0.9261
0.9318
0.9449
0.9482
0.9522
0.9526
0.9549
0.9584
0.9589
0.9599
0.9633
0.9635
0.9644
0.9658
0.9662
0.9661
0.9667
0.966
0.9669
0.967
0.9705
0.9679
0.9692
0.9698
0.9691
0.9698
0.9703
0.9727
0.972
0.9723
0.97
0.9699
0.9711
0.9716
0.9716
0.9723
0.9736
0.9716
0.9728
0.9732
0.9742
0.9742
0.9747
0.9745
0.976
0.9759
0.9736


In [ ]:
#Processing on GPU

#sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

steps = 5000
sess.run(init)
for step in range(steps):
    batch_x,batch_y = mnist.train.next_batch(100)

    sess.run(train, feed_dict={X:batch_x,y_true:batch_y,hold_prob:0.5})
    # PRINT OUT A MESSAGE EVERY 100 STEPS
    #print('started step {}'.format(step))
    if step%100 == 0:
        print('Currently on step {}'.format(step))
        print('Accuracy is:')
        matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
        acc = tf.reduce_mean(tf.cast(matches,dtype=tf.float32))

        print(sess.run(acc,feed_dict = {X : mnist.test.images, y_true : mnist.test.labels,hold_prob:1.0}))
    
    

In [67]:
acc

<tf.Tensor 'Mean_121:0' shape=() dtype=float32>